In [ ]:
import vertexai
from vertexai.language_models import TextEmbeddingModel, TextEmbeddingInput

PROJECT_ID = "rthallam-demo-project"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")
embedding = embedding_model.get_embeddings(["The cat slept soundly on the soft rug."])

vector = embedding[0].values
print(f"Length = {len(vector)}")
print(vector[:5])

In [ ]:

reviews = [
    """I had to compare two versions of Hamlet for my Shakespeare class and \
unfortunately I picked this version. Everything from the acting (the actors \
deliver most of their lines directly to the camera) to the camera shots (all \
medium or close up shots...no scenery shots and very little back ground in the \
shots) were absolutely terrible. I watched this over my spring break and it is \
very safe to say that I feel that I was gypped out of 114 minutes of my \
vacation. Not recommended by any stretch of the imagination.
Sentiment of the message: negative""",

"""Something surprised me about this movie - it was actually original. It was not \
the same old recycled crap that comes out of Hollywood every month. I saw this \
movie on video because I did not even know about it before I saw it at my \
local video store. If you see this movie available - rent it - you will not \
regret it.
Sentiment of the message: positive""",

"""My family has watched Arthur Bach stumble and stammer since the movie first \
came out. We have most lines memorized. I watched it two weeks ago and still \
get tickled at the simple humor and view-at-life that Dudley Moore portrays. \
Liza Minelli did a wonderful job as the side kick - though I\'m not her \
biggest fan. This movie makes me just enjoy watching movies. My favorite scene \
is when Arthur is visiting his fiancée\'s house. His conversation with the \
butler and Susan\'s father is side-spitting. The line from the butler, \
"Would you care to wait in the Library" followed by Arthur\'s reply, \
"Yes I would, the bathroom is out of the question", is my NEWMAIL \
notification on my computer.
Sentiment of the message: positive""",

"""This Charles outing is decent but this is a pretty low-key performance. Marlon \
Brando stands out. There\'s a subplot with Mira Sorvino and Donald Sutherland \
that forgets to develop and it hurts the film a little. I\'m still trying to \
figure out why Charlie want to change his name.
Sentiment of the message: negative"""
]

In [ ]:
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")
input_reviews = [TextEmbeddingInput(task_type="CLASSIFICATION", text=review) for review in reviews]
embedding_reviews = embedding_model.get_embeddings(texts=input_reviews)


In [ ]:
import pandas as pd
import numpy as np

embedding_reviews = [np.array(e.values) for e in embedding_reviews]
dict_reviews = dict(reviews=reviews, embeddings=embedding_reviews)
df_reviews = pd.DataFrame.from_dict(dict_reviews)

In [ ]:
df_reviews.head()

In [35]:
def get_similar_reviews(review: str, vector_store: pd.DataFrame, sort_index_value: int=2):
    review_input = [TextEmbeddingInput(task_type="CLASSIFICATION", text=review)]
    review_vector = embedding_model.get_embeddings(texts=review_input)
    vector_store["dot_product"] = vector_store["embeddings"].apply(lambda row: np.dot(row, review_vector))
    top_matched = vector_store.sort_values(by="dot_product", ascending=False)[:sort_index_value].index
    top_matched_df = vector_store.loc[top_matched, ["reviews"]]
    context = "\n".join(top_matched_df["reviews"].values)
    return context

In [36]:
review = """Tweet: The Pixel 7 Pro, is too big to fit in my jeans pocket, so I bought \
new jeans.
Sentiment of the message: """

get_similar_reviews(review, df_reviews)

ValueError: shapes (768,) and (1,) not aligned: 768 (dim 0) != 1 (dim 0)

# Multilingual embeddings

In [37]:
import vertexai
from vertexai.language_models import TextEmbeddingModel, TextEmbeddingInput

PROJECT_ID = "rthallam-demo-project"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

embedding_model = TextEmbeddingModel.from_pretrained("text-multilingual-embedding-002")
embedding = embedding_model.get_embeddings(["休士頓哪裡有正宗的葡式蛋撻和港式麵包"])

vector = embedding[0].values
print(f"Length = {len(vector)}")
print(vector[:5])

Length = 768
[0.04212726652622223, 0.0060269092209637165, -0.041167981922626495, 0.033627621829509735, -0.013369345106184483]


# Multimodal embeddings

In [43]:
VMImage?

Init signature: VMImage(image_bytes: Optional[bytes] = None, gcs_uri: Optional[str] = None)
Docstring:      Image.
Init docstring:
Creates an `Image` object.

Args:
    image_bytes: Image file bytes. Image can be in PNG or JPEG format.
    gcs_uri: Image URI in Google Cloud Storage.
File:           /opt/conda/envs/genai-gcp/lib/python3.10/site-packages/vertexai/vision_models/__init__.py
Type:           type
Subclasses:     GeneratedImage

In [65]:
mm_embedding_model.get_embeddings?

Signature:
mm_embedding_model.get_embeddings(
    image: Optional[vertexai.vision_models.Image] = None,
    video: Optional[vertexai.vision_models.Video] = None,
    contextual_text: Optional[str] = None,
    dimension: Optional[int] = None,
    video_segment_config: Optional[vertexai.vision_models.VideoSegmentConfig] = None,
) -> 'MultiModalEmbeddingResponse'
Docstring:
Gets embedding vectors from the provided image.

Args:
    image (Image): Optional. The image to generate embeddings for. One of
      `image`, `video`, or `contextual_text` is required.
    video (Video): Optional. The video to generate embeddings for. One of
      `image`, `video` or `contextual_text` is required.
    contextual_text (str): Optional. Contextual text for your input image or video.
      If provided, the model will also generate an embedding vector for the
      provided contextual text. The returned image and text embedding
      vectors are in the same semantic space with the same dimensionality,
   

In [71]:
import vertexai
from vertexai.vision_models import (
    Image as VMImage,
    MultiModalEmbeddingModel,
    MultiModalEmbeddingResponse,
    Video as VMVideo,
    VideoSegmentConfig,
)

PROJECT_ID = "rthallam-demo-project"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

mm_embedding_model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding")

# Image embeddings with default 1408 dimension
image_path = "gs://github-repo/embeddings/getting_started_embeddings/gms_images/GGOEACBA104999.jpg"
image_path = image_path.replace("gs://", "https://storage.googleapis.com/").replace(" ", "%20")
image = VMImage.load_from_file(image_path)
dimension = 256
image_embedding = mm_embedding_model.get_embeddings(image=image, dimension=dimension).image_embedding

print("length of embedding: ", len(image_embedding))
print("First five values are: ", image_embedding[:5])

length of embedding:  256
First five values are:  [0.170544401, 0.00523724314, 0.0162872504, -0.00227135723, 0.0331991129]


In [46]:
image_bytes

NameError: name 'image_bytes' is not defined

In [62]:
import requests

def read_image_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.content
    else:
        raise Exception(f"Failed to download image: {response.status_code}")

image_url = "https://upload.wikimedia.org/wikipedia/commons/1/19/Solar_System_true_color.jpg"
image_bytes = read_image_from_url(image_url)
# image = VMImage.load_from_file(image_bytes)
image_embedding = mm_embedding_model.get_embeddings(image=image, dimension=dimension).image_embedding

In [63]:
len(image_embedding)

256